# IceNet Data Analysis

## Context

### Purpose
The IceNet library provides the ability to download, process, train and predict from end to end via a set of command-line interfaces.

Using this notebook one can understand the various data sources, intermediaries and products that arise from the [CLI demonstrator notebook](01.cli_demonstration.ipynb) activities.

### Modelling approach
This modelling approach allows users to immediately utilise the library for producing sea ice concentraion forecasts.

### Highlights
The key features of an end to end run are: 
* [Setup](#Setup)
* [Download](#Download) 
* [Process](#Process)
* [Train](#Train)
* [Predict](#Predict)

_This follows the same structure as the CLI demonstration notebook so that it's easy to follow step-by-step..._

### Contributions
#### Notebook
James Byrne (author)

__Please raise issues [in this repository](https://github.com/antarctica/IceNet-Pipeline) to suggest updates to this notebook!__ 

Contact me at _jambyr \<at\> bas.ac.uk_ for anything else...

#### Modelling codebase
James Byrne (code author), Tom Andersson (science author)

#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

#### Involved organisations
The Alan Turing Institute and British Antarctic Survey

In [ ]:
## Setup

In [ ]:
## Download

In [ ]:
## Process

In [ ]:
## Train

In [ ]:
## Predict


## Plotting a forecast

TODO: convert

```python
from icenet2.plotting.video import xarray_to_video as xv
import xarray as xr
ds = xr.open_dataset("south_daily_forecast.nc")
fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
fc['time'] = [dt.datetime(2022,1,3) + dt.timedelta(days=int(e)) for e in fc.time.values]
xv(fc, 15, "south_daily_forecast.mp4")
```


## Video processing of datasets

```bash
icenet_video_data -w 4 -v osisaf
icenet_video_data -w 4 -n -sy -p processed/north_10 -v era5,osisaf
```

```
import glob, os, xarray as xr

for vn in ("zg250", "zg500", "tos"):
    dfs = glob.glob("data/era5/sh/{}/**/19*.nc".format(vn)) + glob.glob("data/era5/sh/{}/**/20*.nc".format(vn))
    for y in range(1979, 2020):
        ds = [d for d in dfs if os.path.basename(d).startswith(str(y))]
        print("{} {} {} files".format(vn, str(y), len(ds)))
        if len(ds) > 0:
            ds2 = xr.open_mfdataset(ds, parallel=True)
            k = list(ds2.data_vars)[0]
            d_min, d_max = getattr(ds2, k).min(), getattr(ds2, k).max()
            print("\t\t\t{} {} {:.2f} {:.2f}".format(y, k, float(d_min.compute()), float(d_max.compute())))
    
    
```

## Version
- Notebook:
- Codebase: